# Unmapped Metabase

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Catalog/Unmapped/data'
STORE = 'alianza'

## Import

In [2]:
import sys
sys.path.append('/Users/efraflores/Desktop/hub/cornershop/venv/lib/python3.9/site-packages')

In [3]:
import os
import time
import numpy as np
import pandas as pd
from IPython.lib.display import Audio

prod = pd.read_csv(os.path.join(BASE_DIR,f'prod_{STORE}.csv')).drop_duplicates()
supply = pd.read_csv(os.path.join(BASE_DIR,f'supply_{STORE}.csv')).drop_duplicates()

display(prod.sample())
display(supply.sample())

,store_id,catalog_product_id,branch,productbranch_id,price
119197,98,1898031,941,58463445,131.0


,sku_source,supply_product_id,catalog_product_id,name,barcode,branch,price_mode,stock,min_last_visit
5390,mx-bodegasalianza-csv,14137653,4093829.0,Sidra alianza ambar,7.503009e+12,7657.0,62.0,52,2021-06-25T16:30:58.834762Z


## Functions

### Timing and tone

In [4]:
start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

### Batches

In [5]:
def batches(base_dir,store,data,size=10000):
    df = data.copy()
    n = 0
    for i in range(int(np.ceil(len(df)/size))):
        batch = df.iloc[n:n+size-1,:]
        batch.to_excel(os.path.join(base_dir,f'bulk_{store}_{str(i+1).zfill(2)}.xlsx'),index=False)
        n += size
    print(f'The file with {len(df)} rows has been split up into {i+1} batches of {size} rows each')

### Summary

In [6]:
def summary():
    print(f"Para la tienda {STORE.title()}, los productbranch se distribuyen:\
    \n-Catálogo\t{len(prod)}\
    \n-Supply\t\t{len(supply)}\
    \n-Sin mapear\t{len(unmapped)+len(brands)}\t({'{:.1%}'.format((len(unmapped)+len(brands))/len(full))})\
    \n -Brands\t{len(branch_brands)}\
    \n -Sin órdenes\t{len(df[df['order_products']==0])}\
    \n -Buen perform\t{len(good_perf)}\t(FR {'{:.1%}'.format(good_perf['fr'].mean())})\
    \n -Mal perform\t{len(bad_fr)}\t(FR {'{:.1%}'.format(bad_fr['fr'].mean())})\
    \n -Markup +-3%\t{len(mk)}\t(MK {int(mk['mk_tot'].sum())}, {'{:.1%}'.format(mk['mk_tot'].sum()/mk['sales'].sum())})\
    \n\n-Por desactivar\t{len(bad_perf)}\t({'{:.1%}'.format(len(bad_perf)/len(prod))})\
    \n -Productos \t{len(bad_perf.groupby('catalog_product_id')['store_id'].count())}\
    \n -Orders_prod\t{int(bad_perf['order_products'].sum())}\t({'{:.1%}'.format(bad_perf['order_products'].sum()/total['order_products'].sum())})\
    \n -Ventas $ \t{int(bad_perf['sales'].sum())}\t({'{:.1%}'.format(bad_perf['sales'].sum()/total['sales'].sum())})")

## Unmapped from prod

In [7]:
full = prod.merge(supply,how='left',on=['branch','catalog_product_id'])
full.isnull().mean()

store_id              0.000000
catalog_product_id    0.000000
branch                0.000000
productbranch_id      0.000000
price                 0.000000
sku_source            0.591167
supply_product_id     0.591167
name                  0.591167
barcode               0.591167
price_mode            0.591167
stock                 0.591167
min_last_visit        0.591167
dtype: float64

In [8]:
unmapped = full[full['supply_product_id'].isnull()].dropna(axis=1)
#unmapped.to_csv(os.path.join(BASE_DIR,f'check_brands_{STORE}.csv'),index=False)

## Brands

Check with @Alan Sánchez Márquez if any product_id was created by Brands

In [9]:
brands = pd.read_csv(os.path.join(BASE_DIR,f'brands.csv'))
brands = brands[brands['Integración']=='Sin mapear'].copy()
brands.sample(3)

,Product ID,Origen,Barcode (EAN),Nombre del producto,Marca,Package (Gramaje),Fecha de creación,Días de vida,Precio,Store,Integración,NOTAS,Gsheet KAM,Gsheet terminado,PBI,PBI2,Status Final
1635,334700,Editado,7500435108249,Detergente lavatrastes limón,Salvo,1400 ml,15/12/2020,-196.0,$94.90,HEB,Sin mapear,"REVISAR EL PRECIO, ya que en competencia el p...",NaN,NaN,NaN,NaN,Desactivado.
2951,1230473,Editado,7503017964318,Clusters de Granola con cobertura de Crema de ...,Berry Nuts,Bolsa 180g,5/2/2021,-144.0,$52.20,Fresko,Sin mapear,NaN,NaN,NaN,NaN,NaN,Desactivado.
6039,1246514,Editado,7501318626553,Solución nimotop,Bayer,1 Caja con 1 frasco de 50 ml,2/6/2021,-27.0,$657.00,Soriana,Sin mapear,NaN,https://docs.google.com/spreadsheets/d/1KGxhfo...,https://docs.google.com/spreadsheets/d/16-PqHF...,NaN,NaN,Desactivado.


In [10]:
import re
brands['inactive'] = brands['Status Final'].apply(lambda x:re.search('desac',str(x),flags=re.I)!=None)*1
brands.rename(columns={'Product ID':'catalog_product_id'},inplace=True)
brands = brands[(brands['Status Final'].notnull())&(brands['inactive']==0)]
brands[['catalog_product_id','inactive']].sample(4)

,catalog_product_id,inactive
1114,1005347,0
2083,1456280,0
2030,1246057,0
2091,1144240,0


In [11]:
brands[brands['inactive']==0]['Status Final'].value_counts(0,dropna=False)

Activo (mapeado).    39
Reactivado.           2
Store.                1
Buen FR.              1
Name: Status Final, dtype: int64

In [12]:
branch_brands = unmapped[unmapped['catalog_product_id'].isin(brands['catalog_product_id'])]
unmapped = unmapped[~unmapped['catalog_product_id'].isin(brands['catalog_product_id'])].copy()
#','.join(set(unmapped['catalog_product_id'].dropna().astype(int).astype(str)))

## Orders from unmapped

In [13]:
orders = pd.read_csv(os.path.join(BASE_DIR,f'orders_{STORE}.csv'))
orders.head()

,level,date,catalog_product_id,branch,productbranch_id,min_branch_price,avg_branch_price,max_branch_price,min_shopper_price,avg_shopper_price,max_shopper_price,requested,found,order_products,sales
0,TOTAL,2021-06-07,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,166.0,126.0,76,22355.00
1,TOTAL,2021-06-08,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,122.0,109.0,63,27850.50
2,TOTAL,2021-06-09,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,176.0,142.0,88,38453.90
3,TOTAL,2021-06-10,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,123.0,108.0,83,29211.79
4,TOTAL,2021-06-11,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,168.0,149.0,119,48469.79


In [14]:
total = orders[orders['level']=='TOTAL'].set_index('date').iloc[:,-4:].copy()
total['fr'] = total['found']/total['requested']
total

,requested,found,order_products,sales,fr
date,,,,,
2021-06-07,166.0,126.0,76,22355.00,0.759036
2021-06-08,122.0,109.0,63,27850.50,0.893443
2021-06-09,176.0,142.0,88,38453.90,0.806818
2021-06-10,123.0,108.0,83,29211.79,0.878049
2021-06-11,168.0,149.0,119,48469.79,0.886905
2021-06-12,158.0,127.0,106,45692.30,0.803797
2021-06-13,28.0,24.0,23,8637.30,0.857143
2021-06-14,34.0,29.0,25,10713.00,0.852941
2021-06-15,256.0,183.0,77,47852.00,0.714844


### Total

In [15]:
df = unmapped.set_index('catalog_product_id').merge(orders.iloc[:,1:],on='productbranch_id',how='left').fillna(0)
df = df[~df['catalog_product_id'].isin(brands['catalog_product_id'])].copy()
df = df.sort_values('order_products',0,0).reset_index(drop=True)
df['fr'] = (df['found']/(df['requested']+1e-10)).apply(lambda x: min(1,x))
print(df['fr'].describe())
df.head()

count    89096.000000
mean         0.006054
std          0.077492
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: fr, dtype: float64


,store_id,branch_x,productbranch_id,price,date,catalog_product_id,branch_y,min_branch_price,avg_branch_price,max_branch_price,min_shopper_price,avg_shopper_price,max_shopper_price,requested,found,order_products,sales,fr
0,98,7628,45769157,295.0,2021-06-10,1993298.0,7628.0,295.0,295.0,295.0,325.0,325.0,325.0,2.0,1.0,2.0,295.0,0.5
1,98,916,8284666,745.0,2021-06-19,384378.0,916.0,745.0,745.0,745.0,745.0,767.0,789.0,2.0,2.0,2.0,1490.0,1.0
2,98,916,8239069,635.0,2021-06-11,352963.0,916.0,635.0,635.0,635.0,610.0,610.0,610.0,2.0,2.0,2.0,1270.0,1.0
3,98,7651,38194797,69.0,2021-06-24,357346.0,7651.0,69.0,69.0,69.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,98,930,8229270,327.0,2021-06-27,352942.0,930.0,327.0,327.0,327.0,327.0,327.0,327.0,2.0,2.0,1.0,654.0,1.0


### Bad performance

BULK TOOL

<https://cornershopapp.com/catalog/dashboard/bulk_update/product_branches/>

In [16]:
df['catalog_product_id'] = df['catalog_product_id'].astype(int).astype(str)
df = df.groupby(['store_id','productbranch_id',
                 'catalog_product_id']).agg({'price':'median','requested':sum,
                                             'found':sum,'sales':sum,
                                             'order_products':sum}).reset_index()

In [17]:
df['fr'] = (df['found'] / (df['requested']+1e-10)).apply(lambda x: min(x,1))
df['avg_price'] = df['sales'] / df['found']
df['mk_un'] = df['avg_price'] - df['price']
df['mk_p'] = df['mk_un'] / df['avg_price']
df['mk_tot'] = df['sales'] - df['price']*df['found']
bad_fr = df[(df['fr']<=total['fr'][0]*.9)
            #&(df['orders']>0)
           ].copy()
mk = df[(df['mk_p']<-.03)|(df['mk_p']>.03)].copy()
bad_perf = bad_fr.append(mk)
print(bad_perf['fr'].describe())
bad_perf.head()

count    88559.000000
mean         0.000137
std          0.010096
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: fr, dtype: float64


,store_id,productbranch_id,catalog_product_id,price,requested,found,sales,order_products,fr,avg_price,mk_un,mk_p,mk_tot
0,98,8205112,0,77.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
1,98,8205114,0,54.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
2,98,8205116,0,54.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
3,98,8205117,0,36.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
4,98,8205118,340921,68.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,0.0


### Good performance

Ask the KAM to talk to the partner in order to activate that product

In [18]:
good_perf = df[(~df['productbranch_id'].isin(bad_perf['productbranch_id']))&
               (df['order_products']>0)].copy()
good_perf.to_csv(os.path.join(BASE_DIR,f'good_perf_{STORE}.csv'),index=False)
print(good_perf['fr'].describe())
good_perf.head()

count    456.000000
mean       0.999452
std        0.011707
min        0.750000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: fr, dtype: float64


,store_id,productbranch_id,catalog_product_id,price,requested,found,sales,order_products,fr,avg_price,mk_un,mk_p,mk_tot
35,98,8205166,357712,323.0,1.0,1.0,323.0,1.0,1.0,323.0,0.0,0.0,0.0
87,98,8205242,355285,415.0,1.0,1.0,415.0,1.0,1.0,415.0,0.0,0.0,0.0
116,98,8205277,340772,190.0,2.0,2.0,380.0,1.0,1.0,190.0,0.0,0.0,0.0
118,98,8205279,333349,165.0,1.0,1.0,165.0,1.0,1.0,165.0,0.0,0.0,0.0
171,98,8205349,170519,644.0,1.0,1.0,644.0,1.0,1.0,644.0,0.0,0.0,0.0


## Bulk

In [19]:
bad_perf.to_csv(os.path.join(BASE_DIR,f'bad_performance_{STORE}.csv'),index=False)

In [20]:
export = bad_perf[['store_id','productbranch_id']].copy().drop_duplicates()
export.rename(columns={'productbranch_id':'product_branch_id'},inplace=True)
export['price'] = ''
export['availability'] = 'UNAVAILABLE'
export['schedule'] = ''
export.head()

,store_id,product_branch_id,price,availability,schedule
0,98,8205112,,UNAVAILABLE,
1,98,8205114,,UNAVAILABLE,
2,98,8205116,,UNAVAILABLE,
3,98,8205117,,UNAVAILABLE,
4,98,8205118,,UNAVAILABLE,


## Summary

In [21]:
batches(BASE_DIR,STORE,export)

The file with 88559 rows has been split up into 9 batches of 10000 rows each


In [22]:
time_exp(time.time()-start)
tono()

0 minutos con 12.30 segundos


In [23]:
summary()

Para la tienda Alianza, los productbranch se distribuyen:    
-Catálogo	148204    
-Supply		91395    
-Sin mapear	89058	(59.1%)    
 -Brands	0    
 -Sin órdenes	88365    
 -Buen perform	456	(FR 99.9%)    
 -Mal perform	88553	(FR 0.0%)    
 -Markup +-3%	6	(MK -2365, -20.8%)    

-Por desactivar	88559	(59.8%)    
 -Productos 	154    
 -Orders_prod	210	(13.5%)    
 -Ventas $ 	17499	(2.3%)
